## Yahoo Finance Stock Recommendation Analytical Report

In [15]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
from datetime import datetime
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, Image
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib import colors
import os
import io
from PIL import Image as PILImage
from reportlab.lib.units import inch

# --- User Input Section ---
def get_user_inputs():
    """
    Collects financial details from the user.
    """
    risk_level = input("Enter your risk level (Low, Medium, High): ").strip().capitalize()
    while risk_level not in ["Low", "Medium", "High"]:
        print("Invalid risk level. Please choose from Low, Medium, or High.")
        risk_level = input("Enter your risk level (Low, Medium, High): ").strip().capitalize()

    start_date_str = input("Enter the start date (YYYY-MM-DD): ")
    end_date_str = input("Enter the end date (YYYY-MM-DD): ")

    # Validate date inputs
    try:
        start_date = datetime.strptime(start_date_str, '%Y-%m-%d').date()
        end_date = datetime.strptime(end_date_str, '%Y-%m-%d').date()
        if start_date >= end_date:
            raise ValueError("Start date must be earlier than end date.")
    except ValueError as e:
        print(f"Invalid date format or date range: {e}")
        return None  # Indicate failure to get valid dates

    try:
        max_price = float(input("Enter the maximum price per share you are willing to pay: $ "))
        num_stocks = int(input("Enter the number of stocks you want to buy: "))
        months_held = int(input("How many months do you plan to hold onto the stock? "))

        if max_price <= 0 or num_stocks <= 0 or months_held <= 0:
            raise ValueError("Price, number of stocks, and holding period must be positive values.")
    except ValueError as e:
        print(f"Invalid input: {e}")
        return None # Indicate failure to get valid numeric inputs

    dividend_preference = input("Do you prefer dividend-paying stocks? (Yes/No): ").strip().lower()
    while dividend_preference not in ["yes", "no"]:
        print("Invalid response. Please answer 'Yes' or 'No'.")
        dividend_preference = input("Enter your risk level (Yes/No): ").strip().lower()

    return {
        "risk_level": risk_level,
        "start_date": start_date,
        "end_date": end_date,
        "max_price": max_price,
        "num_stocks": num_stocks,
        "dividend_preference": dividend_preference,
        "months_held": months_held
    }

# --- Stock Recommendation Function ---
def recommend_stocks(risk_level, max_price, dividend_preference, stock_universe):
    """
    Recommends stocks based on risk level, price, and dividend preference using a pre-defined stock universe.
    """
    stock_data = {}
    valid_tickers = [] # List to store tickers for which data is successfully retrieved

    # Fetch stock data using yfinance for each stock
    for ticker in stock_universe:
        try:
            stock = yf.Ticker(ticker)
            info = stock.info
            stock_data[ticker] = info
            valid_tickers.append(ticker)  # Add to the list of valid tickers
        except Exception as e:
            print(f"Could not retrieve data for {ticker}: {e}")
            continue

    # Risk thresholds for different risk levels using beta as a risk measure
    risk_thresholds = {
        "Low": lambda beta: beta < 1,
        "Medium": lambda beta: 1 <= beta <= 1.5,
        "High": lambda beta: beta > 1.5
    }

   # Filter stocks based on risk level and other criteria
    selected = [ticker for ticker, data in stock_data.items()
                if ticker in valid_tickers and  # Only process tickers for which data was successfully retrieved
                risk_thresholds.get(risk_level)(data.get("beta", 1)) and  # Apply risk level criteria
                data.get("currentPrice", float('inf')) <= max_price]  # Check maximum price

    # Filter stocks based on dividend preference
    if dividend_preference == "yes":
        selected = [ticker for ticker in selected if stock_data[ticker].get("dividendYield", 0) > 0]

    return selected

# --- Stock Data Fetching Function ---
def get_stock_data(ticker, start_date, end_date):
    """
    Fetches historical stock data from yfinance.
    """
    try:
        stock = yf.Ticker(ticker)
        data = stock.history(start=start_date, end=end_date)
        return data['Close'] if not data.empty else None
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None

# --- Financial Analysis Functions ---
def calculate_financial_metrics(stock_df):
    """
    Calculates financial metrics such as daily returns, volatility, and Sharpe ratio.
    """
    # Calculate daily returns, volatility, and Sharpe ratio
    daily_returns = stock_df.pct_change().dropna()
    volatility = daily_returns.std() * np.sqrt(252)  # Annualize volatility
    sharpe_ratios = daily_returns.mean() / daily_returns.std() * np.sqrt(252)  # Annualize Sharpe ratio

    return daily_returns, volatility, sharpe_ratios

def get_additional_stock_metrics(selected_stocks):
    """
    Retrieves additional stock metrics such as P/E ratio, dividend yield, and market cap.
    """
    pe_ratios = {}
    dividend_yields = {}
    current_prices = {}
    pb_ratios = {}
    roe_ratios = {}
    market_caps = {}

    for ticker in selected_stocks:
        try:
            stock = yf.Ticker(ticker)
            info = stock.info
            pe_ratios[ticker] = info.get('trailingPE', 'N/A')
            dividend_yields[ticker] = info.get('dividendYield', 0)
            current_prices[ticker] = info.get('currentPrice', 0)
            pb_ratios[ticker] = info.get('priceToBook', 'N/A')
            roe_ratios[ticker] = info.get('returnOnEquity', 'N/A')
            market_caps[ticker] = info.get('marketCap', 0)
        except Exception as e:
            print(f"Could not retrieve additional metrics for {ticker}: {e}")
            # Set default values in case of error to avoid breaking the program
            pe_ratios[ticker] = 'N/A'
            dividend_yields[ticker] = 0
            current_prices[ticker] = 0
            pb_ratios[ticker] = 'N/A'
            roe_ratios[ticker] = 'N/A'
            market_caps[ticker] = 0

    return pe_ratios, dividend_yields, current_prices, pb_ratios, roe_ratios, market_caps

def calculate_investment_metrics(selected_stocks, dividend_yields, current_prices, months_held, num_stocks, max_price):
    """
    Calculates investment metrics such as dividend earnings and net profits.
    """
    dividends_in_usd = {}
    net_profits = {}
    total_investment = max_price * num_stocks

    for ticker in selected_stocks:
        # Monthly dividend earnings calculation (annual yield * price * months held / 12 * number of stocks)
        dividends_in_usd[ticker] = (dividend_yields[ticker] * current_prices[ticker] * months_held / 12) * num_stocks

        # Calculate total investment cost for each stock
        investment_cost = current_prices[ticker] * num_stocks

        # Net profit (dividends - investment cost) for each stock
        net_profit = dividends_in_usd[ticker] - investment_cost
        net_profits[ticker] = net_profit

    return dividends_in_usd, net_profits, total_investment

def perform_stock_growth_analysis(stock_df, selected_stocks):
    """
    Analyzes stock growth, optimal buy/sell days, and holding time.
    """
    stock_growth_data = {}
    optimal_buy_sell_data = {}
    calculation_steps = {}

    for ticker in selected_stocks:
        calculation_steps[ticker] = {}

        # 1. Stock Growth Calculation
        initial_price = stock_df[ticker].iloc[0]
        final_price = stock_df[ticker].iloc[-1]
        growth = ((final_price - initial_price) / initial_price) * 100
        stock_growth_data[ticker] = growth

        calculation_steps[ticker]['stock_growth'] = [
            f"Stock Growth Calculation for {ticker}:",
            f"Formula: ((Final Price - Initial Price) / Initial Price) * 100",
            f"Final Price: {final_price:.2f}",
            f"Initial Price: {initial_price:.2f}",
            f"Calculation: (({final_price:.2f} - {initial_price:.2f}) / {initial_price:.2f}) * 100 = {growth:.2f}%",
            f"Stock Growth: {growth:.2f}%"
        ]

        # 2. Optimal Buy and Sell Days (Identify historical best entry/exit points)
        min_price = stock_df[ticker].min()
        max_price = stock_df[ticker].max()
        best_buy_date = stock_df[ticker][stock_df[ticker] == min_price].index[0].strftime('%Y-%m-%d')
        best_sell_date = stock_df[ticker][stock_df[ticker] == max_price].index[0].strftime('%Y-%m-%d')

        optimal_buy_sell_data[ticker] = {
            "Best Buy Date": best_buy_date,
            "Best Buy Price": min_price,
            "Best Sell Date": best_sell_date,
            "Best Sell Price": max_price
        }

        calculation_steps[ticker]['optimal_buy_sell'] = [
            "Optimal Buy and Sell Days:",
            f"Best Day to Buy: {best_buy_date} (Price: ${min_price:.2f})",
            f"Best Day to Sell: {best_sell_date} (Price: ${max_price:.2f})"
        ]

    return stock_growth_data, optimal_buy_sell_data, calculation_steps

def calculate_holding_time(start_date, end_date):
    """
    Calculates the average holding time in days.
    """
    holding_period = (end_date - start_date).days
    average_holding_time = holding_period
    return average_holding_time

def predict_future_price(ticker, current_price, historical_growth):
    """
    Predicts the future stock price based on current price and historical growth.
    """
    predicted_price = current_price * (1 + (historical_growth / 100))  # Apply growth
    return predicted_price

def perform_profit_comparison(stock_df, selected_stocks, total_investment, dividend_yields, current_prices, months_held, net_profits, num_stocks, stock_growth_data):
    """
    Compares profit potential if bought at the lowest price versus buying today, and includes a growth projection.
    """
    profit_comparison_data = {}
    calculation_steps = {}

    for ticker in selected_stocks:
        calculation_steps[ticker] = {}

        # 4a. Historical Profit Projection (Bought at Lowest)
        lowest_price = stock_df[ticker].min()
        shares_at_lowest = total_investment / lowest_price  # Invest total investment at the lowest price

        # Total profit calculation: (shares at lowest * current price) + dividend earnings - initial investment
        profit_at_lowest = (shares_at_lowest * current_prices[ticker]) + ((dividend_yields[ticker] * current_prices[ticker] * months_held / 12) * shares_at_lowest) - total_investment

        calculation_steps[ticker]['profit_at_lowest'] = [
            f"Historical Profit Projection (Bought at Lowest - Price: ${lowest_price:.2f}):",
            f"Formula: (Shares at Lowest * Current Price) + (Dividend Earnings) - Total Investment",
            f"Shares at Lowest: Total Investment / Lowest Price = ${total_investment:.2f} / ${lowest_price:.2f} = {shares_at_lowest:.2f} shares",
            f"Dividend Yield: {dividend_yields[ticker]:.4f}",
            f"Current Price: ${current_prices[ticker]:.2f}",
            f"Dividend Earnings: (Dividend Yield * Current Price * Months Held / 12) * Shares at Lowest = ({dividend_yields[ticker]:.4f} * ${current_prices[ticker]:.2f} * {months_held} / 12) * {shares_at_lowest:.2f} = ${(dividend_yields[ticker] * current_prices[ticker] * months_held / 12) * shares_at_lowest:.2f}",
            f"Profit: (Shares at Lowest * Current Price) + Dividend Earnings - Total Investment = ({shares_at_lowest:.2f} * ${current_prices[ticker]:.2f}) + ${(dividend_yields[ticker] * current_prices[ticker] * months_held / 12) * shares_at_lowest:.2f} - ${total_investment:.2f} = ${profit_at_lowest:.2f}",
            f"Potential Profit if Bought at Lowest: ${profit_at_lowest:.2f}"
        ]

        # 4b. Projected Profit (Zero Growth Scenario)
        profit_today = net_profits[ticker]

        calculation_steps[ticker]['profit_today'] = [
            f"Projected Profit (Zero Growth Scenario - Price: ${current_prices[ticker]:.2f}):",
            f"Projected Profit if Bought Today: ${profit_today:.2f}",
            f"Investment Cost: ${current_prices[ticker] * num_stocks:.2f}, Number of stocks: {num_stocks}",
            f"Dividend Earning = ((dividend_yields[ticker] * current_prices[ticker]) * Months Held/12)*Number of stocks = ((({dividend_yields[ticker]:.4f} * {current_prices[ticker]}) * {months_held}/12)*{num_stocks:.2f}) = ${ (dividend_yields[ticker] * current_prices[ticker] * months_held / 12) * num_stocks:.2f}",
            f"Projected Profit if Bought Today: Dividend Earning - Investment Cost =  ${(dividend_yields[ticker] * current_prices[ticker] * months_held / 12) * num_stocks:.2f} - ${current_prices[ticker] * num_stocks:.2f} = {profit_today:.2f}",
            "Calculation Details: See 'Net Profit (USD)' in Stock Performance Summary above."
        ]

        # 4c. Projected Profit (Growth Scenario)
        historical_growth = stock_growth_data[ticker]  # historical_growth as an average annual growth
        if historical_growth > 0:

            predicted_future_price = predict_future_price(ticker, current_prices[ticker], historical_growth) #future price is based on historical growth

            #Recalculate Profit
            future_profit =  (dividend_yields[ticker] * predicted_future_price * months_held / 12) * num_stocks - (current_prices[ticker] * num_stocks) # predicted dividend earnings - current cost
            
            calculation_steps[ticker]['future_profit'] = [
                f"Projected Profit (Growth Scenario - Predicted Future Price: ${predicted_future_price:.2f}):",
                f"The future profit = (Annual dividend yield * predicted_future_price * Holding months / 12)*number of stocks - Current Price.  ",
                f"The Future Projections ${future_profit:.2f}"
            ]
            
        elif historical_growth <= 0: #If the stock is expected to decrease
            calculation_steps[ticker]['future_profit'] = [
                "Stock is predicted to reduce in value, no projections available, see zero growth scenario!", #Displaying the zero projection again to prevent overestimations
                f"The Zero growth projection is ${profit_today:.2f}" #Zero growth
            ]

        profit_comparison_data[ticker] = {
            "Historical Profit Projection (Bought at Lowest)": profit_at_lowest,
            "Projected Profit (Zero Growth Scenario)": profit_today,
            "Projected Profit (Growth Scenario)": future_profit if historical_growth > 0 else "N/A"
        }

    return profit_comparison_data, calculation_steps

def monte_carlo_simulation(stock_df, daily_returns, num_simulations=50000):
    """
    Performs a Monte Carlo simulation to forecast potential stock prices.
    """
    num_trading_days = len(stock_df)
    simulations = {}  # Store simulations for each ticker

    for ticker in stock_df.columns:
        # Generate random daily returns based on the historical distribution
        mean_return = daily_returns[ticker].mean()
        std_dev = daily_returns[ticker].std()

        # Start the simulation from the last known price
        last_price = stock_df[ticker].iloc[-1]

        # Initialize a simulation array for the ticker
        simulated_prices = [last_price]  # Start with the last actual price

        for _ in range(num_trading_days):
            # Generate a random daily return
            random_return = np.random.normal(mean_return, std_dev)

            # Calculate the next day's price
            next_price = simulated_prices[-1] * (1 + random_return)

            # Append the price to the simulation array
            simulated_prices.append(next_price)

        # Store the simulation results
        simulations[ticker] = simulated_prices[:num_trading_days]

    return simulations

def generate_monte_carlo_plot(simulation_data, ticker):
    """
    Generates a plot of the Monte Carlo simulation results for a specific ticker.
    """
    pio.renderers.default = "png"

    fig = go.Figure()
    fig.add_trace(go.Scatter(y=simulation_data, mode='lines', name=ticker))

    fig.update_layout(title=f'Monte Carlo Simulation for {ticker}',
                      xaxis_title='Trading Days',
                      yaxis_title='Stock Price',
                      template='plotly_white')

    # Convert the plot to a PNG image
    img_bytes = fig.to_image(format="png")
    img = PILImage.open(io.BytesIO(img_bytes))
    return img

def generate_stock_report(filename, user_inputs, selected_stocks, stock_data, stock_growth_data, optimal_buy_sell_data, profit_comparison_data, calculation_steps, monte_carlo_simulations, additional_metrics):
    """
    Generates a PDF report containing the stock analysis.
    """
    doc = SimpleDocTemplate(filename, pagesize=letter)
    styles = getSampleStyleSheet()
    story = []

    # --- Title and User Inputs ---
    story.append(Paragraph("Stock Recommendation Report", styles['h1']))
    story.append(Paragraph(f"Risk Level: {user_inputs['risk_level']}", styles['Normal']))
    story.append(Paragraph(f"Start Date: {user_inputs['start_date']}", styles['Normal']))
    story.append(Paragraph(f"End Date: {user_inputs['end_date']}", styles['Normal']))
    story.append(Paragraph(f"Max Price per Share: ${user_inputs['max_price']}", styles['Normal']))
    story.append(Paragraph(f"Number of Stocks: {user_inputs['num_stocks']}", styles['Normal']))
    story.append(Paragraph(f"Months Held: {user_inputs['months_held']}", styles['Normal']))
    story.append(Paragraph(f"Dividend Preference: {user_inputs['dividend_preference']}", styles['Normal']))
    story.append(Spacer(1, 12))

    # --- Stock Recommendations ---
    story.append(Paragraph("Stock Recommendations:", styles['h2']))
    if selected_stocks:
        story.append(Paragraph(f"Based on your criteria, the following stocks are recommended: {', '.join(selected_stocks)}", styles['Normal']))
    else:
        story.append(Paragraph("No stocks match your criteria.", styles['Normal']))
    story.append(Spacer(1, 12))

    # --- Stock Performance Summary Table ---
    story.append(Paragraph("Stock Performance Summary", styles['h2']))
    table_data = [['Ticker', 'Current Price', 'P/E Ratio', 'Dividend Yield', 'Market Cap', 'P/B Ratio', 'ROE']]
    for ticker in selected_stocks:
        current_price = additional_metrics[2][ticker]
        pe_ratio = additional_metrics[0][ticker]
        dividend_yield = additional_metrics[1][ticker]
        market_cap = additional_metrics[5][ticker]
        pb_ratio = additional_metrics[3][ticker]
        roe = additional_metrics[4][ticker]
        table_data.append([ticker, f'${current_price:.2f}', str(pe_ratio), f'{dividend_yield:.4f}', f'${market_cap:,.2f}', str(pb_ratio), str(roe)])

    table = Table(table_data)
    table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
        ('GRID', (0, 0), (-1, -1), 1, colors.black)
    ]))
    story.append(table)
    story.append(Spacer(1, 12))

    # --- Detailed Analysis for Each Stock ---
    for ticker in selected_stocks:
        story.append(PageBreak())
        story.append(Paragraph(f"Detailed Analysis for {ticker}", styles['h2']))

        # Profit Projection Table
        # Check if Projected Profit (Growth Scenario) is a number, otherwise, use "N/A"
        growth_profit = profit_comparison_data[ticker]["Projected Profit (Growth Scenario)"]
        if isinstance(growth_profit, (int, float)):
            growth_profit_str = f'${growth_profit:.2f}'
        else:
            growth_profit_str = "N/A"

        profit_table_data = [
            ['Scenario', 'Projected Profit'],
            ['Historical (Bought at Lowest)', f'${profit_comparison_data[ticker]["Historical Profit Projection (Bought at Lowest)"]:.2f}'],
            ['Zero Growth', f'${profit_comparison_data[ticker]["Projected Profit (Zero Growth Scenario)"]:.2f}'],
            ['Growth Scenario', growth_profit_str]
        ]

        profit_table = Table(profit_table_data)
        profit_table.setStyle(TableStyle([
            ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
            ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
            ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
            ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
            ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
            ('GRID', (0, 0), (-1, -1), 1, colors.black)
        ]))
        story.append(profit_table)
        story.append(Spacer(1, 12))

        # Stock Growth Analysis
        story.append(Paragraph("Stock Growth Analysis:", styles['h3']))
        for step in calculation_steps[ticker].get('stock_growth', []):
            story.append(Paragraph(step, styles['Normal']))
        story.append(Spacer(1, 6))

        # Optimal Buy and Sell Days
        story.append(Paragraph("Optimal Buy and Sell Days:", styles['h3']))
        for step in calculation_steps[ticker].get('optimal_buy_sell', []):
            story.append(Paragraph(step, styles['Normal']))
        story.append(Spacer(1, 6))

        # Profit Comparison
        story.append(Paragraph("Profit Comparison:", styles['h3']))

        # Profit at Lowest
        story.append(Paragraph("Historical Profit Projection (Bought at Lowest):", styles['h4']))
        for step in calculation_steps[ticker].get('profit_at_lowest', []):
            story.append(Paragraph(step, styles['Normal']))
        story.append(Spacer(1, 6))

        # Zero Growth Scenario
        story.append(Paragraph("Projected Profit (Zero Growth Scenario):", styles['h4']))
        for step in calculation_steps[ticker].get('profit_today', []):
            story.append(Paragraph(step, styles['Normal']))
        story.append(Spacer(1, 6))

        # Growth Scenario
        story.append(Paragraph("Projected Profit (Growth Scenario):", styles['h4']))
        steps = calculation_steps[ticker].get('future_profit', [])
        for step in steps:
            story.append(Paragraph(step, styles['Normal']))
        story.append(Spacer(1, 6))

        # --- Monte Carlo Simulation ---
        story.append(Paragraph("Monte Carlo Simulation:", styles['h3']))

        # Generate and include Monte Carlo plot
        simulation_data = monte_carlo_simulations[ticker]  # Get simulation data for the ticker
        img = generate_monte_carlo_plot(simulation_data, ticker) # Pass the ticker data
        img_io = io.BytesIO()
        img.save(img_io, format='PNG')
        img_reportlab = Image(img_io, width=5 * inch, height=3 * inch)
        story.append(img_reportlab)
        story.append(Spacer(1, 6))

        # Monte Carlo Simulation Table Data
        mean_simulation_price = np.mean(simulation_data)
        median_simulation_price = np.median(simulation_data)
        std_dev_simulation = np.std(simulation_data)
        min_simulation_price = np.min(simulation_data)
        max_simulation_price = np.max(simulation_data)

        table_data = [
            ['Metric', 'Value'],
            ['Mean Simulated Price', f'${mean_simulation_price:.2f}'],
            ['Median Simulated Price', f'${median_simulation_price:.2f}'],
            ['Standard Deviation', f'${std_dev_simulation:.2f}'],
            ['Minimum Simulated Price', f'${min_simulation_price:.2f}'],
            ['Maximum Simulated Price', f'${max_simulation_price:.2f}']
        ]
        table = Table(table_data)
        table.setStyle(TableStyle([
            ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
            ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
            ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
            ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
            ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
            ('GRID', (0, 0), (-1, -1), 1, colors.black)
        ]))
        story.append(table)
        story.append(Spacer(1, 6))

        # Monte Carlo Simulation Calculations
        story.append(Paragraph("Monte Carlo Simulation Calculations:", styles['h4']))
        story.append(Paragraph(f"Mean Simulated Price Calculation: Average of all simulated prices = ${mean_simulation_price:.2f}", styles['Normal']))
        story.append(Paragraph(f"Median Simulated Price Calculation: Middle value of all simulated prices = ${median_simulation_price:.2f}", styles['Normal']))
        story.append(Paragraph(f"Standard Deviation Calculation: Measure of the spread of simulated prices = ${std_dev_simulation:.2f}", styles['Normal']))
        story.append(Paragraph(f"Minimum Simulated Price: Lowest simulated price = ${min_simulation_price:.2f}", styles['Normal']))
        story.append(Paragraph(f"Maximum Simulated Price: Highest simulated price = ${max_simulation_price:.2f}", styles['Normal']))
        story.append(Spacer(1, 12))

    doc.build(story)

# --- Main Execution ---
if __name__ == "__main__":
    # 1. Define the stock universe
    stock_universe = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "META", "NFLX", "NVDA", "SPY", "QQQ",
        "IBM", "WMT", "GE", "DIS", "BA", "V", "JNJ", "PFE", "KO", "XOM", "CVX", "INTC",
        "T", "CSCO", "ORCL", "BABA", "NKE", "AMD", "SBUX", "GS", "HD", "MCD", "LULU",
        "PYPL", "UBER", "SQ", "WFC", "CRM", "SHOP", "TSM", "SNAP", "BA", "LMT", "CAT",
        "RTX", "ZION", "AMAT", "INTU", "MU", "ATVI", "RCL", "UAL", "F", "GE", "VZ", "MS",
        "MDT", "CVX", "MCK", "TMO", "BNS", "C", "AIG", "BMY", "MA", "GS", "FISV", "TGT",
        "PFE", "HPE", "LNC", "T", "DHR", "PFE", "NVS", "AZN", "SNE", "HLT", "VEEV", "BA",
        "AIG", "CI", "HUM", "AMT", "VZ", "LLY", "ABBV", "VLO", "PSX", "SYY", "PG", "CLX",
        "PEP", "KO", "CVS", "WBA", "MCK", "MSFT", "GOOG", "PFE", "MGM", "UAL", "RTX", "EXC",
        "ABBV", "AAPL", "NVDA", "AMD", "FB", "DIS", "LULU", "ACN", "NKE", "INTC", "GS", "MS",
        "COF", "STZ", "UNH", "MDT", "XOM", "ETSY", "FDX", "INTU", "TMO", "PFG", "APD", "EXR",
        "SBUX", "ABT", "JNJ", "STZ", "NXPI", "CL", "WMT", "PBA", "NXST", "LNT", "TSCO", "EMR",
        "SWKS", "WDC", "WFC", "AMGN", "BLK", "V", "ITW", "HCA", "AON", "NEE", "DHR", "TMO",
        "CHTR", "FIS", "WBA", "MU", "AAPL", "TGT", "COST", "AMZN", "BABA", "HD", "TRV", "MRK",
        "VLO", "COP", "SIVB", "GM", "F", "X", "GM", "CVX", "CVS", "FISV", "SYY", "NEE",
        "INTC", "BA", "GS", "SBUX", "MCD", "WMT", "PYPL", "BABA", "TSLA", "GOOG", "T", "AMZN",
        "AMAT", "MA", "SNE", "TMO", "C", "PYPL", "BA", "V", "VZ", "EBAY", "ACN", "CSCO",
        "BBY", "LULU", "WFC", "ETSY", "TGT", "ORCL", "CVX", "PFE", "NKE", "GE", "BMY", "KO",
        "ADBE", "SPLK", "AIG", "AMZN", "EXC", "TMO", "MRK", "NVDA", "BA", "SYY", "JPM", "IBM",
        "AAPL", "AMD", "MSFT", "TSLA", "WMT", "PG", "CVS", "SBUX", "V", "BABA", "KO", "XOM",
        "SPY", "TSM", "ACN", "CSCO", "FB", "PYPL", "NFLX", "VZ", "PG", "MA", "KO", "LLY",
        "PYPL", "DIS", "VEEV", "SBUX", "MMM", "LMT", "PFE", "NKE", "VZ", "COST", "NDAQ", "MS",
        "GM", "ATVI", "MS", "LMT", "TGT", "PLD", "WBA", "GOOG", "BABA", "CSX", "EA", "COP",
        "PFE", "BNSF", "PG", "HCA", "COF", "AXP", "DELL", "T", "TMO", "UNH", "BMY", "AZN",
        "HUM", "CVX", "NSC", "BIDU", "FCX", "GE", "HAL", "K", "MCK", "DD", "AVGO", "GS", "KHC",
        "KO", "MMM", "MCD", "SBUX", "BA", "CAT", "TGT", "WFC", "MSFT", "FDX", "AIG", "V",
        "IBB", "XLF", "SPY", "IWM", "EEM", "VWO", "SCHD", "MSCI", "VOO", "ACWI", "VTI", "F",
        "PYPL", "PFE", "DIS", "UPS", "TGT", "WMT", "FDX", "NEE", "UNP", "HD", "COST", "SWK",
        "NKE", "SCHW", "BIIB", "VLO", "EXC", "DD", "PH", "UNH", "HUM", "NVDA", "AVGO", "COF",
        "AAPL", "META", "MSFT", "GOOG", "XOM", "CVX", "SPY", "AMZN", "NFLX", "TGT", "TSLA",
        "AIG", "AMT", "IBM", "GE", "STZ", "GILD", "PFE", "AMD", "INTC", "BA", "MMM", "F", "SBUX"]

    # 2. Get user inputs
    user_inputs = get_user_inputs()
    if user_inputs is None:
        print("Invalid inputs. Please try again.")
    else:
        # 3. Recommend stocks based on user inputs
        selected_stocks = recommend_stocks(user_inputs["risk_level"], user_inputs["max_price"],
                                            user_inputs["dividend_preference"], stock_universe)

        if selected_stocks:
            # 4. Fetch stock data for the selected stocks
            stock_data = {}
            for ticker in selected_stocks:
                stock_data[ticker] = get_stock_data(ticker, user_inputs["start_date"], user_inputs["end_date"])

            # Filter out tickers with no data
            stock_data = {ticker: data for ticker, data in stock_data.items() if data is not None}
            selected_stocks = list(stock_data.keys())  # Update selected_stocks based on available data

            if selected_stocks:
                stock_df = pd.DataFrame(stock_data)

                # 5. Calculate financial metrics
                daily_returns, _, _ = calculate_financial_metrics(stock_df)

                # 6. Get additional stock metrics
                additional_metrics = get_additional_stock_metrics(selected_stocks)
                pe_ratios, dividend_yields, current_prices, pb_ratios, roe_ratios, market_caps = additional_metrics

                # 7. Calculate investment metrics
                dividends_in_usd, net_profits, total_investment = calculate_investment_metrics(
                    selected_stocks, dividend_yields, current_prices, user_inputs["months_held"],
                    user_inputs["num_stocks"], user_inputs["max_price"]
                )

                # 8. Perform stock growth analysis
                stock_growth_data, optimal_buy_sell_data, growth_calculation_steps = perform_stock_growth_analysis(stock_df, selected_stocks)

                # 9. Perform profit comparison
                profit_comparison_data, profit_calculation_steps = perform_profit_comparison(
                    stock_df, selected_stocks, total_investment, dividend_yields, current_prices,
                    user_inputs["months_held"], net_profits, user_inputs["num_stocks"], stock_growth_data
                )

                # 10. Perform Monte Carlo simulation
                monte_carlo_simulations = monte_carlo_simulation(stock_df, daily_returns)

                # 11. Generate the stock report
                all_calculation_steps = {}
                for ticker in selected_stocks:
                    all_calculation_steps[ticker] = {
                        **growth_calculation_steps[ticker],
                        **profit_calculation_steps[ticker]
                    }

                generate_stock_report("stock_report.pdf", user_inputs, selected_stocks, stock_data,
                                        stock_growth_data, optimal_buy_sell_data, profit_comparison_data,
                                        all_calculation_steps, monte_carlo_simulations, additional_metrics)

                print("Stock report generated successfully: stock_report.pdf")
            else:
                print("No stock data available for the selected stocks.")
        else:
            print("No stocks match your criteria.")

Enter your risk level (Low, Medium, High):  Low
Enter the start date (YYYY-MM-DD):  2020-01-01
Enter the end date (YYYY-MM-DD):  2025-03-04
Enter the maximum price per share you are willing to pay: $  75
Enter the number of stocks you want to buy:  10
How many months do you plan to hold onto the stock?  12
Do you prefer dividend-paying stocks? (Yes/No):  Yes


Could not retrieve data for SQ: list index out of range


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ATVI?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ATVI&crumb=4ztqcap8qFs


Could not retrieve data for ATVI: 'NoneType' object has no attribute 'update'


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/SIVB?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=SIVB&crumb=4ztqcap8qFs


...

Could not retrieve data for SPLK: 'NoneType' object has no attribute 'update'


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ATVI?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ATVI&crumb=4ztqcap8qFs


Could not retrieve data for ATVI: 'NoneType' object has no attribute 'update'


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/BNSF?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=BNSF&crumb=4ztqcap8qFs


Could not retrieve data for BNSF: 'NoneType' object has no attribute 'update'
Stock report generated successfully: stock_report.pdf
